<a href="https://colab.research.google.com/github/dreamerlin/mmaction2/blob/tutorial_ipython/demo/MMAction2_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAction2 Tutorial

Welcome to MMAction2! This is the official colab tutorial for using MMAction2. In this tutorial, you will learn
- Perform inference with a MMAction2 recognizer.
- Train a new recognizer with a new dataset.

Let's start!

## Install MMAction2

In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -e .
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.5.1+cu101 in /usr/local/lib/python3.6/dist-packages (1.5.1+cu101)
Requirement already up-to-date: torchvision==0.6.1+cu101 in /usr/local/lib/python3.6/dist-packages (0.6.1+cu101)
     |████████████████████████████████| 235kB 5.2MB/s 
     |████████████████████████████████| 194kB 15.1MB/s 
  Created wheel for mmcv: filename=mmcv-1.0.2-cp36-cp36m-linux_x86_64.whl size=383271 sha256=414a17a556aa6a3ac3cd3a707fb9df37cbbe587c46591aa3b35c2f614c698760
  Stored in directory: /root/.cache/pip/wheels/89/72/2f/090938947dcf1975a513b99748fcaf0277d18e834cf5d9095a
Successfully built mmcv
Cloning into 'mmaction2'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4736 (delta 0), reused 1 (delta 0), pack-reused 4732
Receiving objects: 100% (4736/4736), 29.67 MiB | 23.26 MiB/s, done.
Resolving deltas: 1

     |████████████████████████████████| 11.8MB 258kB/s 
     |████████████████████████████████| 36.9MB 66kB/s 
  Created wheel for PyTurboJPEG: filename=PyTurboJPEG-1.4.1-cp36-none-any.whl size=7002 sha256=8730aa46378c847a4e69c8e2b6c9ce9ad74c2da31bb2bbd1af8ff9a6b28a7bc5
  Stored in directory: /root/.cache/pip/wheels/c3/21/97/152eed6e60d59f1c432139dee7a2e89de44026ef1855b4c4d7
Successfully built PyTurboJPEG


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

1.5.1+cu101 True
0.1.0+0c57f69


## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [5]:
!mkdir checkpoints
!wget -c https://openmmlab.oss-accelerate.aliyuncs.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2020-07-12 06:23:39--  https://openmmlab.oss-accelerate.aliyuncs.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving openmmlab.oss-accelerate.aliyuncs.com (openmmlab.oss-accelerate.aliyuncs.com)... 47.88.36.1
Connecting to openmmlab.oss-accelerate.aliyuncs.com (openmmlab.oss-accelerate.aliyuncs.com)|47.88.36.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  13.0MB/s    in 7.2s    

2020-07-12 06:23:47 (13.0 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [6]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

In [7]:
# Use the recognizer to do inference
video = 'demo/demo.mp4'
label = 'demo/label_map.txt'
results = inference_recognizer(model, video, label)

In [8]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

arm wrestling:  29.61644
rock scissors paper:  10.754843
shaking hands:  9.908401
clapping:  9.189913
massaging feet:  8.305306


## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [9]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://openmmlab.oss-accelerate.aliyuncs.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2020-07-12 06:24:36--  https://openmmlab.oss-accelerate.aliyuncs.com/mmaction/kinetics400_tiny.zip
Resolving openmmlab.oss-accelerate.aliyuncs.com (openmmlab.oss-accelerate.aliyuncs.com)... 47.88.36.1
Connecting to openmmlab.oss-accelerate.aliyuncs.com (openmmlab.oss-accelerate.aliyuncs.com)|47.88.36.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  4.71MB/s    in 14s     

2020-07-12 06:24:51 (1.23 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [10]:
# Check the directory structure of the tiny data

# Install tree first
!apt-get -q install tree
!tree kinetics400_tiny

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (46.7 kB/s)
Selecting previously unselected package tree.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50

In [11]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [17]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [18]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [19]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Config (path: ./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py): {'model': {'type': 'Recognizer2D', 'backbone': {'type': 'ResNet', 'pretrained': 'torchvision://resnet50', 'depth': 50, 'norm_eval': False}, 'cls_head': {'type': 'TSNHead', 'num_classes': 2, 'in_channels': 2048, 'spatial_type': 'avg', 'consensus': {'type': 'AvgConsensus', 'dim': 1}, 'dropout_ratio': 0.4, 'init_std': 0.01}}, 'train_cfg': None, 'test_cfg': {'average_clips': None}, 'dataset_type': 'VideoDataset', 'data_root': 'kinetics400_tiny/train/', 'data_root_val': 'kinetics400_tiny/val/', 'ann_file_train': 'kinetics400_tiny/kinetics_tiny_train_video.txt', 'ann_file_val': 'kinetics400_tiny/kinetics_tiny_val_video.txt', 'ann_file_test': 'kinetics400_tiny/kinetics_tiny_val_video.txt', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_bgr': False}, 'train_pipeline': [{'type': 'DecordInit'}, {'type': 'SampleFrames', 'clip_len': 1, 'frame_interval': 1, 'num_clips': 8},

2020-07-12 06:28:45,761 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
2020-07-12 06:28:45,808 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2020-07-12 06:28:45,911 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2020-07-12 06:28:45,915 - mmaction - INFO - Start running, host: root@ec77c2666f1a, work_dir: /content/mmaction2/tutorial_exps
2020-07-12 06:28:45,915 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
2020-07-12 06:28:51,422 - mmaction - INFO - Epoch [1][5/15]	lr: 7.813e-05, eta: 0:08:09, time: 1.100, data_time:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.4 task/s, elapsed: 2s, ETA:     0s

2020-07-12 06:29:34,236 - mmaction - INFO - Evaluating top_k_accuracy...
2020-07-12 06:29:34,238 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2020-07-12 06:29:34,239 - mmaction - INFO - Evaluating mean_class_accuracy...
2020-07-12 06:29:34,243 - mmaction - INFO - 
mean_acc	0.7000
2020-07-12 06:29:34,244 - mmaction - INFO - Now best checkpoint is epoch_5.pth
2020-07-12 06:29:34,246 - mmaction - INFO - Epoch [5][15/15]	lr: 7.813e-05, top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2020-07-12 06:29:39,685 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:03:56, time: 1.085, data_time: 0.656, memory: 2918, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.6653, loss: 0.6653, grad_norm: 9.9450
2020-07-12 06:29:41,676 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:03:47, time: 0.399, data_time: 0.006, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls: 0.6991, loss: 0.6991, grad_norm: 11.1495
2020-07-12 06:29:43,541 - mmaction - INFO - Epoch 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.4 task/s, elapsed: 2s, ETA:     0s

...

2020-07-12 06:32:01,071 - mmaction - INFO - Evaluating top_k_accuracy...
2020-07-12 06:32:01,072 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2020-07-12 06:32:01,075 - mmaction - INFO - Evaluating mean_class_accuracy...
2020-07-12 06:32:01,077 - mmaction - INFO - 
mean_acc	1.0000
2020-07-12 06:32:01,080 - mmaction - INFO - Now best checkpoint is epoch_20.pth
2020-07-12 06:32:01,083 - mmaction - INFO - Epoch [20][15/15]	lr: 7.813e-05, top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
2020-07-12 06:32:06,592 - mmaction - INFO - Epoch [21][5/15]	lr: 7.813e-05, eta: 0:01:29, time: 1.101, data_time: 0.682, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.5018, loss: 0.5018, grad_norm: 9.8059
2020-07-12 06:32:08,608 - mmaction - INFO - Epoch [21][10/15]	lr: 7.813e-05, eta: 0:01:26, time: 0.403, data_time: 0.004, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.4527, loss: 0.4527, grad_norm: 8.2144
2020-07-12 06:32:10,493 - mmaction - INFO - Epo

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.5 task/s, elapsed: 2s, ETA:     0s

2020-07-12 06:32:50,062 - mmaction - INFO - Evaluating top_k_accuracy...
2020-07-12 06:32:50,064 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2020-07-12 06:32:50,068 - mmaction - INFO - Evaluating mean_class_accuracy...
2020-07-12 06:32:50,071 - mmaction - INFO - 
mean_acc	1.0000
2020-07-12 06:32:50,072 - mmaction - INFO - Epoch [25][15/15]	lr: 7.813e-05, top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
2020-07-12 06:32:55,581 - mmaction - INFO - Epoch [26][5/15]	lr: 7.813e-05, eta: 0:00:43, time: 1.100, data_time: 0.685, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3789, loss: 0.3789, grad_norm: 8.0383
2020-07-12 06:32:57,568 - mmaction - INFO - Epoch [26][10/15]	lr: 7.813e-05, eta: 0:00:40, time: 0.398, data_time: 0.005, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls: 0.6657, loss: 0.6657, grad_norm: 13.0251
2020-07-12 06:32:59,450 - mmaction - INFO - Epoch [26][15/15]	lr: 7.813e-05, eta: 0:00:36, time: 0.377, data_time: 0.001, memo

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.4 task/s, elapsed: 2s, ETA:     0s

2020-07-12 06:33:39,570 - mmaction - INFO - Evaluating top_k_accuracy...
2020-07-12 06:33:39,572 - mmaction - INFO - 
top1_acc	0.9000
top5_acc	1.0000
2020-07-12 06:33:39,575 - mmaction - INFO - Evaluating mean_class_accuracy...
2020-07-12 06:33:39,580 - mmaction - INFO - 
mean_acc	0.9000
2020-07-12 06:33:39,581 - mmaction - INFO - Epoch [30][15/15]	lr: 7.813e-05, top1_acc: 0.9000, top5_acc: 1.0000, mean_class_accuracy: 0.9000


### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,
 
Not bad!

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.eval_config
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')